In [1]:
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict


class GemRagEngine:
    def __init__(self, model=None):
        """
        model: エンベディングモデル（encodeメソッドを持つもの）
        """
        self.model = model
        self.chunks: List[str] = []
        self.embeddings: np.ndarray | None = None
        self.eigen_scores: np.ndarray | None = None
        self.graph: nx.Graph | None = None

    def ingest(self, texts: List[str], similarity_threshold: float = 0.5):
        """
        ドキュメントを取り込み、グラフを構築して固有値スコアを計算する
        """
        self.chunks = texts

        # 1. エンベディング生成
        print("Creating embeddings...")
        if self.model:
            self.embeddings = np.asarray(self.model.encode(texts))
        else:
            # モデルがない場合はテスト用にランダムベクトルを使用
            np.random.seed(42)
            self.embeddings = np.random.rand(len(texts), 384)

        # 2. 類似度行列 (Similarity Matrix) の計算
        sim_matrix = cosine_similarity(self.embeddings)

        # 3. グラフ構築 (Adjacency Matrix)
        # 閾値以下のリンクを切り、対角成分(自分自身)を0にする
        adj_matrix = (sim_matrix > similarity_threshold).astype(float) * sim_matrix
        np.fill_diagonal(adj_matrix, 0.0)

        # NetworkXグラフに変換（重みは 'weight' 属性に入る）
        self.graph = nx.from_numpy_array(adj_matrix)

        # 4. 固有ベクトル中心性 (Eigen-Memory-like Encoding)
        print("Calculating Eigen-scores...")
        try:
            centrality = nx.eigenvector_centrality(
                self.graph,
                max_iter=1000,
                weight="weight",
                tol=1e-6,
            )
            self.eigen_scores = np.array([centrality[i] for i in range(len(texts))])
        except nx.PowerIterationFailedConvergence:
            print("⚠️ 固有値計算が収束しませんでした。次数ベースのスコアを使用します。")
            degrees = np.array([deg for _, deg in self.graph.degree(weight="weight")])
            if degrees.max() > 0:
                self.eigen_scores = degrees / degrees.max()
            else:
                self.eigen_scores = np.ones(len(texts))

        # スコアの正規化（扱いやすくするため）
        if np.max(self.eigen_scores) > 0:
            self.eigen_scores = self.eigen_scores / np.max(self.eigen_scores)

        print(f"✅ Indexed {len(texts)} chunks.")

    def search(self, query: str, top_k: int = 3, alpha: float = 0.6) -> List[Dict]:
        """
        alpha: 混合比率 (0.0=EigenScoreのみ, 1.0=類似度のみ)
        Score = α * Similarity + (1 - α) * EigenScore
        """
        assert self.embeddings is not None, "ingest() を先に呼んでください。"

        # クエリのベクトル化
        if self.model:
            query_vec = self.model.encode([query])[0]
        else:
            # ★ 修正ポイント: 次元は埋め込みの「特徴次元」だけにする
            np.random.seed(0)
            query_vec = np.random.rand(self.embeddings.shape[1])

        # 1. 類似度スコア (Relevance)
        sim_scores = cosine_similarity([query_vec], self.embeddings)[0]

        # 2. ハイブリッドスコア計算
        final_scores = alpha * sim_scores + (1.0 - alpha) * self.eigen_scores

        # 上位取得
        top_indices = np.argsort(final_scores)[::-1][:top_k]

        results: List[Dict] = []
        for idx in top_indices:
            results.append({
                "chunk": self.chunks[idx],
                "score": float(final_scores[idx]),
                "similarity": float(sim_scores[idx]),
                "eigen_score": float(self.eigen_scores[idx]),
            })

        return results

In [2]:
from sentence_transformers import SentenceTransformer

# 日本語対応の軽量モデル
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
engine = GemRagEngine(model)

# サンプルデータ：RAGに関する断片的な知識
docs = [
    "RAG (Retrieval-Augmented Generation) はLLMに外部知識を与える技術だ。",  # 0: 概要
    "RAGの実装には、一般的にベクトルデータベースが使用される。",            # 1: 技術詳細
    "ベクトル検索では、コサイン類似度を用いて関連文書を探す。",              # 2: 技術詳細
    "GEM-RAGは、グラフ理論を用いてRAGの精度を向上させる手法である。",        # 3: 発展手法
    "GEM-RAGでは、文書間の関係性から固有値を計算し、重要度とする。",          # 4: 3の詳細
    "今日の夕飯はカレーライスにしようと思う。",                              # 5: 完全なノイズ
]

# インデックス作成 (閾値を少し低めに設定してリンクを作りやすくする)
engine.ingest(docs, similarity_threshold=0.3)

print("\n--- 各チャンクの固有値スコア (情報の重要度) ---")
for i, score in enumerate(engine.eigen_scores):
    print(f"ID {i}: {score:.4f} | {docs[i][:20]}...")

print("\n--- 検索テスト: 'GEM-RAGの仕組みは？' ---")
results = engine.search("GEM-RAGの仕組みは？", top_k=3, alpha=0.4)
for res in results:
    print(f"Score: {res['score']:.4f} (Sim: {res['similarity']:.2f}, Eigen: {res['eigen_score']:.2f})")
    print(f"Content: {res['chunk']}")
    print("-" * 20)

c:\Users\LoNebula\miniconda3\envs\bs2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating embeddings...
Calculating Eigen-scores...
✅ Indexed 6 chunks.

--- 各チャンクの固有値スコア (情報の重要度) ---
ID 0: 0.9359 | RAG (Retrieval-Augme...
ID 1: 0.9765 | RAGの実装には、一般的にベクトルデータ...
ID 2: 0.6383 | ベクトル検索では、コサイン類似度を用いて...
ID 3: 0.8743 | GEM-RAGは、グラフ理論を用いてRA...
ID 4: 1.0000 | GEM-RAGでは、文書間の関係性から固...
ID 5: 0.0000 | 今日の夕飯はカレーライスにしようと思う。...

--- 検索テスト: 'GEM-RAGの仕組みは？' ---
Score: 0.8887 (Sim: 0.72, Eigen: 1.00)
Content: GEM-RAGでは、文書間の関係性から固有値を計算し、重要度とする。
--------------------
Score: 0.8310 (Sim: 0.77, Eigen: 0.87)
Content: GEM-RAGは、グラフ理論を用いてRAGの精度を向上させる手法である。
--------------------
Score: 0.7155 (Sim: 0.32, Eigen: 0.98)
Content: RAGの実装には、一般的にベクトルデータベースが使用される。
--------------------


In [3]:
# 1. テキストファイルを読み込む
with open("story.txt", encoding="utf-8") as f:
    text = f.read()

# 段落ごとに分割（「空行」で区切る想定）
raw = text.split("\n\n")
docs = [p.strip() for p in raw if p.strip()]

# 2. エンジン準備
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
engine = GemRagEngine(model)

# 3. インデックス作成
engine.ingest(docs, similarity_threshold=0.3)

print("\n--- 各チャンクの固有値スコア (情報の重要度) ---")
for i, score in enumerate(engine.eigen_scores):
    print(f"ID {i}: {score:.4f} | {docs[i][:20]}...")

print("\n--- 検索テスト: 'GEM-RAGの仕組みは？' ---")
results = engine.search("GEM-RAGの仕組みは？", top_k=3, alpha=0.4)
for res in results:
    print(f"Score: {res['score']:.4f} (Sim: {res['similarity']:.2f}, Eigen: {res['eigen_score']:.2f})")
    print(f"Content: {res['chunk']}")
    print("-" * 20)

Creating embeddings...
Calculating Eigen-scores...
✅ Indexed 170 chunks.

--- 各チャンクの固有値スコア (情報の重要度) ---
ID 0: 0.4701 | # 10,000字程度の物語
text_...
ID 1: 0.0972 | 最後の配達ルートは、いつもの住宅街。三十...
ID 2: 0.6410 | 一軒一軒の表札を見るだけで、そこに住む人...
ID 3: 0.1776 | 角を曲がると、古い喫茶店がある。ここの店...
ID 4: 0.8894 | 「おお、今日で最後だってね。寂しくなるよ...
ID 5: 0.9406 | 「こちらこそ、お世話になりました」...
ID 6: 0.2858 | 林さんは私の前にコーヒーを置いた。「三十...
ID 7: 0.9165 | 「あっという間でした」...
ID 8: 0.7075 | 「嘘つけ。いろんなことがあっただろう」...
ID 9: 0.1720 | 確かに、林さんの言う通りだ。この三十年、...
ID 10: 0.2170 | 十年前、一人息子が大学に合格した。その合...
ID 11: 0.3827 | 五年前、妻が病気で倒れた。入院中、私は毎...
ID 12: 0.6657 | ポストに手紙を入れながら、私は思い出して...
ID 13: 0.1365 | 結婚式の招待状を届けた家族が、数年後に出...
ID 14: 0.5222 | 配達を終えて郵便局に戻ろうとしたとき、カ...
ID 15: 0.5895 | 差出人の欄を見ると、「藤原健一」という名...
ID 16: 0.8443 | 「宛先不明で返送するしかないですね」と若...
ID 17: 0.7280 | 「少し調べてみます」と私は言った。同僚は...
ID 18: 0.3859 | 退職する私に、これ以上の仕事を押し付ける...
ID 19: 0.2040 | 私は古い配達記録を調べ始めた。郵便局の倉...
ID 20: 0.6336 | 私はアパートを訪ねた。管理人室のドアをノ...
ID 21: 0.7216 | 「白石さんねえ……ああ、思い出した。確か...
ID 22: 0.7751 | 「何か、手がかりはありませんか？彼女のこ...